In [ ]:
#for project 2
#copypaste

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')
#https://dataverse.nl/api/access/datafile/354098
#https://www.rug.nl/ggdc/docs/pwt90.dta this is the one in the original.(I think the difference in url style is just the difference between the most recent one and previous editions)

#just loop the names of countries?
fr = 'France'
de = 'Germany'
cn = 'Canada'
it = 'Italy'
jp = 'Japan'
uk = 'United Kingdom'
us = 'United States'

country = [fr, de, cn, it, jp, uk, us]
end_data = pd.DataFrame()

#これとかを入れる？
num = 0

for i in country:
    #I don't know if the isin(country[i]) part will work.
    #got the indicies must be integers or slices not strings error again here. isin(country[i])

    #if i = 1
        #とか？めっちゃ非効率的だけど
    #    data = pwt1001.loc[pwt1001['country'].isin('France')][['year', 'countrycode', 'rgdpna', 'rnna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
    #    data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

    #this seems to work but unsure of results
    cc = country[num]

    #  !!!!!! rnna !!!!!!   NOT rkna

    #still error with [num]    only list-like objects are allowed to be passed to isin(), you passed a [str]
        #works now
    data = pwt1001.loc[pwt1001['country'].isin([cc])][['year', 'countrycode', 'rgdpna', 'rnna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
    data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

    #maybe can optimize by using loops here with a list for country names, but leave till late
    #多分行列計算？
    #dataframeに列を追加するのにはappendは必要ない。
    data['alpha'] = 1 - data['labsh']
    data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
    data['hours'] = data['emp'] * data['avh']  # L
    data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
    data['cap_term'] = (data['rnna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
    data['lab_term'] = data['hours'] / data['pop']  # L/N
    data = data.sort_values('year').groupby('countrycode').apply(lambda x: x.assign(
        alpha=1 - x['labsh'],
        y_n_shifted=100 * x['y_n'] / x['y_n'].iloc[0],
        tfp_term_shifted=100 * x['tfp_term'] / x['tfp_term'].iloc[0],
        cap_term_shifted=100 * x['cap_term'] / x['cap_term'].iloc[0],
        lab_term_shifted=100 * x['lab_term'] / x['lab_term'].iloc[0]
    )).reset_index(drop=True).dropna()

    #need to do the chart.
    #
    #data['pop_absolute'] = data['pop'] * 1000000
    #data['y/pop'] = data['rgdpna'] / data['pop_absolute']

    #(1)
    data['y/l'] = data['rgdpna'] / data['hours']
    data['k/l'] = data['rnna'] / data['hours']
    #(2)
    data['Ak^a'] = data['rtfpna'] * data['k/l'] ** data['alpha']
    data['lny'] = np.log(data['y/l'])
    data['lnA'] = np.log(data['rtfpna'])
    data['lnk'] = np.log(data['k/l'])
    #(3)
    #微分って増加率で良いんだよね？
    data['dlny'] = data['lny'].diff()
    data['dlnA'] = data['lnA'].diff()
    data['dlnk'] = data['lnk'].diff()
    av_y = data['dlny'].sum()
    av_a = data['dlnA'].sum()
    av_k = data['dlnk'].sum()
    alpha = data['alpha'].mean()
    a_1 = 1 - alpha

    print([cc, av_total, av_a, av_k, a_1, alpha])


    # Print first 10 rows of data
    #print(data.head(10))

    # Plot data
    #plt.plot(data['year'], data['y_n_shifted'], label='y_n_shifted')
    #plt.plot(data['year'], data['tfp_term_shifted'], label='tfp_term_shifted')
    #plt.plot(data['year'], data['cap_term_shifted'], label='cap_term_shifted')
    #plt.plot(data['year'], data['lab_term_shifted'], label='lab_term_shifted')
    #plt.legend()
    #plt.title(cc)
    #plt.show()

    num = num + 1

print(end_data)
